# TextGeneration 

## Prétraitement des Tweets

In [2]:
import re
import string
from typing import List, Tuple
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Téléchargement des ressources NLTK 
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## preprocess_tweet(tweet: str) -> str

Cette fonction prend en entrée un tweet et le prétraite en supprimant les emojis, les mentions d'utilisateur, les hashtags, les liens URL, les chiffres et la ponctuation.

In [38]:
def preprocess_tweet(tweet: str) -> str:
    """
    Prétraitement d'un tweet en supprimant les emojis, les mentions d'utilisateur,
    les hashtags, les liens URL, les chiffres et la ponctuation.
    """
    # Supprimer les emojis
    tweet = re.sub(r'[^\w\s,]', '', tweet)
    
    # Supprimer les mentions d'utilisateur (@username)
    tweet = re.sub(r'@[A-Za-z0-9_]+', '', tweet)
    
    # Supprimer les hashtags (#hashtag)
    tweet = re.sub(r'#([^\s]+)', '', tweet)
    
    # Supprimer les liens URL (http://...)
    tweet = re.sub(r'http\S+', '', tweet)
    
    # Supprimer les chiffres
    tweet = re.sub(r'\d+', '', tweet)
    
    # Supprimer la ponctuation
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    
    return tweet


## tokenize_tweet(tweet: str) -> list

Cette fonction tokenise un tweet en utilisant TweetTokenizer de NLTK.

In [39]:
def tokenize_tweet(tweet: str) -> list:
    """
    Tokenisation d'un tweet en utilisant TweetTokenizer de NLTK.
    """
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(tweet)
    return tokens



## remove_stopwords(tokens: list) -> list

Cette fonction supprime les mots vides de la liste de tokens en anglais.

In [40]:
def remove_stopwords(tokens: list) -> list:
    """
    Suppression des mots vides de la liste de tokens.
    """
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens

## lemmatize_tokens(tokens: list) -> list

Cette fonction lemmatise les tokens en utilisant WordNetLemmatizer de NLTK.

In [41]:
def lemmatize_tokens(tokens: list) -> list:
    """
    Lemmatisation des tokens en utilisant WordNetLemmatizer de NLTK.
    """
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

## preprocess_tweets_from_file(file_path: str) -> list

Cette fonction prétraite les tweets à partir d'un fichier.

In [42]:
def preprocess_tweets_from_file(file_path: str) -> list:
    """
    Prétraitement des tweets à partir d'un fichier.
    """
    preprocessed_tweets = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            tweet = line.strip()
            preprocessed_tweet = preprocess_tweet(tweet)
            tokens = tokenize_tweet(preprocessed_tweet)
            filtered_tokens = remove_stopwords(tokens)
            lemmatized_tokens = lemmatize_tokens(filtered_tokens)
            preprocessed_tweets.append(lemmatized_tokens)
    
    return preprocessed_tweets

In [43]:
def process_data(data: str) -> Tuple[List[str], List[List[str]]]:
    """
    Tokenize les données en phrases et en mots, en utilisant le prétraitement des tweets.

    Paramètres :
        data (str) : Les données d'entrée sous forme de chaîne de caractères.

    Retourne :
        Tuple[List[str], List[List[str]]] : Un tuple contenant la liste des phrases et la liste des phrases tokenisées.
    """
    def tokenizer_phrase(phrase: str) -> List[str]:
        return preprocess_tweet(phrase).split()

    phrases = data.split('\n')
    phrases = [phrase.strip() for phrase in phrases if phrase.strip()]
    phrases_tokenisees = [tokenizer_phrase(p) for p in phrases]
    
    return phrases, phrases_tokenisees

In [44]:
file_path = "C:\\Users\\user\\Downloads\\23032024125908292688468987\\PA_files\\big_data_chunk_1.txt"


with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read()
phrases, phrases_tokenisees = process_data(data)
print("\nPhrases tokenisées :") : 

for tokens in phrases_tokenisees:
    print(tokens)


Phrases tokenisées :
['How', 'are', 'you', 'Btw', 'thanks', 'for', 'the', 'RT', 'You', 'gonna', 'be', 'in', 'DC', 'anytime', 'soon', 'Love', 'to', 'see', 'you', 'Been', 'way', 'way', 'too', 'long']
['When', 'you', 'meet', 'someone', 'special', 'youll', 'know', 'Your', 'heart', 'will', 'beat', 'more', 'rapidly', 'and', 'youll', 'smile', 'for', 'no', 'reason']
['theyve', 'decided', 'its', 'more', 'fun', 'if', 'I', 'dont']
['So', 'Tired', 'D', 'Played', 'Lazer', 'Tag', 'Ran', 'A', 'LOT', 'D', 'Ughh', 'Going', 'To', 'Sleep', 'Like', 'In', 'Minutes']
['Words', 'from', 'a', 'complete', 'stranger', 'Made', 'my', 'birthday', 'even', 'better']
['First', 'Cubs', 'game', 'ever', 'Wrigley', 'field', 'is', 'gorgeous', 'This', 'is', 'perfect', 'Go', 'Cubs', 'Go']
['i', 'no', 'i', 'get', 'another', 'day', 'off', 'from', 'skool', 'due', 'to', 'the', 'wonderful', 'snow', 'and', 'THIS', 'wakes', 'me', 'updamn', 'thing']
['Im', 'coo', 'Jus', 'at', 'work', 'hella', 'tired', 'r', 'u', 'ever', 'in', 'cali'

## Fonctions de Construction du Modèle 

In [45]:
from collections import defaultdict

def count_n_grams(data: List[List[str]], n: int) -> dict:
    """
    Compte tous les n-grammes dans les données fournies.
    
    Paramètres :
        data (list[list[str]]) : Liste de listes de mots.
        n (int) : nombre de mots dans une séquence (n-gramme).
    
    Retourne :
        n_grams (dict) : Un dictionnaire qui mappe un tuple de n mots à sa fréquence.
    """
    n_grams = defaultdict(int)

    for sentence in data:
        sentence = ['<s>']*(n-1) + sentence + ['</s>']
        for i in range(len(sentence) - n + 1):
            n_grams[tuple(sentence[i:i+n])] += 1
    
    return n_grams




In [46]:
def train_ngram_model(data: List[List[str]], ngram_size: int, k: float) -> dict:
    """
    Entraîne un modèle de n-gramme avec lissage add-k à partir des données fournies.
    """
    n_grams = count_n_grams(data, ngram_size)
    vocab_size = len(set(word for sentence in data for word in sentence))

    # Calcul des probabilités avec lissage add-k et normalisation
    ngram_probs = {}
    for ngram, count in n_grams.items():
        context = " ".join(ngram[:-1])
        word = ngram[-1]
        context_count = sum(value for key, value in n_grams.items() if key[:-1] == ngram[:-1])
        prob = (count + k) / (context_count + k * vocab_size)
        
        if context in ngram_probs:
            ngram_probs[context][word] = prob 
        else:
            ngram_probs[context] = {word: prob }

    return ngram_probs


ngram_size = 2
k = 0.01
ngram_model = train_ngram_model(phrases_tokenisees, ngram_size, k)


## Fonctions d'Évaluation du Modèle

In [47]:
import math

def predict_ngram(sentence: str, ngram_model: dict, ngram_size: int) -> float:
    """
    Calcule la probabilité d'une phrase donnée en utilisant un modèle de n-gramme.

    Paramètres :
        sentence (str) : La phrase dont on veut calculer la probabilité.
        ngram_model (dict) : Le modèle de n-gramme entraîné, contenant les probabilités des n-grammes.
        ngram_size (int) : La taille des n-grammes à utiliser dans le modèle.

    Retourne :
        float : La probabilité logarithmique de la phrase selon le modèle de n-gramme.
    """
    # Normalisation de la phrase
    tokens = sentence.lower().split()
    n = len(tokens)

    total_log_prob = 0.0
    for i in range(ngram_size - 1, n):
        context = " ".join(tokens[i - ngram_size + 1:i])
        word = tokens[i]
        if context in ngram_model and word in ngram_model[context]:
            total_log_prob += math.log(ngram_model[context][word])
        else:
            pass

    return total_log_prob



In [48]:
def calculate_perplexity(test_data: List[List[str]], ngram_model: dict, ngram_size: int) -> float:
    """
    Calcule la perplexité d'un modèle de langue n-gramme sur un ensemble de données de test.

    Paramètres :
        test_data (List[List[str]]) : Les données de test sous forme de listes de listes de mots.
        ngram_model (dict) : Le modèle de langue n-gramme entraîné.
        ngram_size (int) : La taille des n-grammes utilisée dans le modèle.

    Retourne :
        float : La perplexité calculée.
    """
    total_log_prob = 0.0
    total_words = 0

    for sentence in test_data:
        sentence = ['<s>']*(ngram_size-1) + sentence + ['</s>']
        for i in range(ngram_size - 1, len(sentence)):
            context = " ".join(sentence[i - ngram_size + 1:i])
            word = sentence[i]
            if context in ngram_model and word in ngram_model[context]:
                total_log_prob += math.log(ngram_model[context][word])
            else:
                pass
                
            total_words += 1
    
    # Calcul de la perplexité
    perplexity = math.exp(-total_log_prob / total_words)
    return perplexity

In [49]:
# Calcul de la perplexité sur les données de test
test_file_path = "C:\\Users\\user\\Downloads\\23032024125908292688468987\\PA_files\\big_data_chunk_3.txt"  
with open(test_file_path, 'r', encoding='utf-8') as file:
    test_data = process_data(file.read())[1]  

perplexity = calculate_perplexity(test_data, ngram_model, ngram_size)
print(f"Perplexity on test data: {perplexity:.4f}")

Perplexity on test data: 8.9206


# Génération de Texte à l'Aide d'un Modèle de Langue N-gramme


# generateText()

Cette fonction génère du texte en utilisant un modèle de langue n-gramme préalablement entraîné. Elle prend en entrée le modèle de n-gramme, la taille des n-grammes utilisée dans le modèle et le nombre maximal de mots à générer (par défaut 100). La génération de texte se fait en sélectionnant itérativement le prochain mot en fonction du contexte actuel et des probabilités du modèle. La fonction s'arrête lorsque le jeton de fin de phrase "</s>" est généré ou lorsque le nombre maximal de mots est atteint.

In [52]:
import numpy as np

def generateText(ngram_model: dict, ngram_size: int, max_words: int = 100) -> str:
    """
    Génère un texte en utilisant un modèle de n-gramme.

    Paramètres :
        ngram_model (dict) : Le modèle de n-gramme entraîné.
        ngram_size (int) : La taille des n-grammes utilisée dans le modèle.
        max_words (int) : Le nombre maximum de mots à générer (par défaut 100).

    Retourne :
        str : Le texte généré.
    """
    generated_text = []
    current_context = ['<s>'] * (ngram_size - 1)

    # Boucle jusqu'à ce que le jeton de fin de phrase </s> soit généré ou que le nombre maximum de mots soit atteint
    while len(generated_text) < max_words:
        context = " ".join(current_context)
        
        # Sélection du prochain mot basé sur le contexte actuel et les probabilités du modèle
        next_word_probs = ngram_model.get(context, {})
        if not next_word_probs:
            break  # Arrêter si le contexte actuel n'a pas de mots suivants dans le modèle

        next_words = list(next_word_probs.keys())
        probabilities = list(next_word_probs.values())
        
        # Normalisation des probabilités pour s'assurer qu'elles s'ajoutent à 1
        total_probability = sum(probabilities)
        normalized_probabilities = [prob / total_probability for prob in probabilities]
        
        # Sélection du mot suivant en utilisant les probabilités normalisées
        next_word = np.random.choice(next_words, p=normalized_probabilities)
        
        # Ajout du mot généré à la liste du texte généré
        generated_text.append(next_word)
        
        # Mise à jour du contexte pour inclure le nouveau mot généré
        current_context.pop(0)
        current_context.append(next_word)
        
        # Arrêter la génération si le jeton de fin de phrase </s> est généré
        if next_word == '</s>':
            break

    return ' '.join(generated_text)


# Texte Généré 

In [54]:
generated_text = generateText(ngram_model, ngram_size=2, max_words=100)
print(generated_text)


Time offer Get here in years </s>
